In [1]:
from time import *
from ephem import *
import math
#
today=strftime("%Y/%m/%d")
horizon='15:00' # can also give in radians,
KAT=Observer()
KAT.long,KAT.lat='21:24:38.46','-30:43:17.34' # positive East,North
KAT.date=today
KAT.epoch=today
# print KAT.epoch
KAT.elev=1038            #metres above MSL
KAT.temp=14             # temperature in C Winter !
KAT.pressure=0         # pressure in mbar set to zero to ignore refraction
KAT.horizon=horizon    # minimum elevation
dd=KAT.date.tuple()
print today

2016/09/27


In [2]:
print "Date %d/%d/%d UT %02d:%02d  LST %s Horizon %s"\
      %(dd[0],dd[1],dd[2],dd[3],dd[4],KAT.sidereal_time(),str(KAT.horizon))
print "Source"+17*" "+"Position"+8*" "+"UT Rise"+2*" "+"UT Transit"+3*" "+"UT Set"+"   Hours Up"
#
sun=Sun()
sun.compute(KAT)
#print "Sun  Alt",  sun.alt, "Az", sun.az
print "Sun          %12s %12s %-9s %-9s %-9s " \
      %(sun.ra,sun.dec,str(KAT.next_rising(sun))[-8:],str(KAT.next_transit(sun))[-8:],str(KAT.next_setting(sun))[-8:])
#
moon=Moon()
moon.compute(KAT)
#print "Moon Alt", moon.alt,"Az", moon.az

print "Moon          %-12s%12s %-9s %-9s %-9s " \
            % (moon.ra,moon.dec,str(KAT.next_rising(moon))[-8:],str(KAT.next_transit(moon))[-8:],str(KAT.next_setting(moon))[-8:])
print 
#
#
#inp=("testsource, J2000, 01:02:14.3213,-11:33:11.9", "test2, B1950, 17:31:43, -83:13:44")
inp=("dec40, J2000,7:00:00,40:00:00", "dec20, J2000, 7:00:00,20:00:00", "dec0, J2000, 7:00:00,00:00:00"\
     , "dec-20, J2000,7:00:00,-20:00:00", "dec-40, J2000,7:00:00,-40:00:00", "dec-60, J2000,7:00:00,-60:00:00")
for line in inp:
  if line[0] != "#":
    s=line.split(",") # split the line into strings
    source=s[0]
# next item is 'radec...'    
    ra=s[2]
    dec=s[3]
    dbline='"'+source+",f|J|Q,"+ra+","+dec+",,2000"+'"'
#    print dbline
    p=readdb(dbline)
#    p=FixedBody()
#    p._ra,p._dec=ra,dec
#    p._epoch='2000'
    p.compute(KAT)
    if p.neverup == True:
       print "%-11s  %12s %12s           Never Rises           " %(source,p.ra,p.dec)
    if p.circumpolar == True:
       transit=str(KAT.next_transit(p))[-8:]
       print "%-11s  %12s %12s          %9s  Circumpolar "  %(source,p.ra,p.dec,transit)
    if p.neverup == False and p.circumpolar == False:  # Source rises and sets
#      print source, p.ra, p.dec, p.alt, p.az,
      rises=str(KAT.next_rising(p))[-8:]
      transit=str(KAT.next_transit(p))[-8:]
      sets=str(KAT.next_setting(p))[-8:]
      uptime=((KAT.next_setting(p)-KAT.next_rising(p))%1)*24  # 
#calculation is in days, so make modulo 1 day, times 24 for hours.
      print "%-12s %12s %12s %-9s %-9s %-9s  %5.2f " % (source,p.ra,p.dec,rises,transit,sets,uptime)
#    for n in range(3,len(s)):    #comments at the end of the input line
#         print s[n],
#    print ""


Date 2016/9/27 UT 00:00  LST 1:50:30.00 Horizon 15:00:00.0
Source                 Position        UT Rise  UT Transit   UT Set   Hours Up
Sun           12:15:50.33   -1:42:46.2 05:29:40  10:25:11  15:21:06  
Moon          9:24:04.07    13:36:46.5 03:26:01  07:45:46  12:09:02  

dec40          7:01:08.74   39:58:19.3 03:22:06  05:09:48  06:57:30    3.59 
dec20          7:00:58.67   19:58:26.4 01:20:00  05:09:38  08:59:16    7.65 
dec0           7:00:50.93   -0:01:26.6 00:20:19  05:09:30  09:58:41    9.64 
dec-20         7:00:43.19  -20:01:20.6 23:30:13  05:09:22  10:44:36   11.24 
dec-40         7:00:33.03  -40:01:16.3 22:42:01  05:09:12  11:32:28   12.84 
dec-60         7:00:13.87  -60:01:14.1 21:24:57  05:08:53  12:48:54   15.40 


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:33: DeprecationWarning: the ephem.Body attributes 'rise_time', 'rise_az', 'transit_time', 'transit_alt', 'set_time', 'set_az', 'circumpolar', and 'never_up' are deprecated; please convert your program to use the ephem.Observer functions next_rising(), previous_rising(), next_transit(), and so forth



In [3]:
print today

2016/09/27


In [4]:
print p.alt,p.az

15:00:00.0 152:07:10.2


In [5]:
KAT

<ephem.Observer date='2016/9/27 00:00:00' epoch='2016/9/27 00:00:00' lon='21:24:38.5' lat='-30:43:17.3' elevation=1038.0m horizon=15:00:00.0 temp=14.0C pressure=0.0mBar>

In [6]:
def LST(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    Set the time string to Hours:Minutes:Seconds
    """
    KAT.date=ut
    lst=str(KAT.sidereal_time())[:8]
    return lst


In [7]:
def JD(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    returns jd
    """
    jd=julian_date(ut)
    return jd

In [8]:
ut=today+" 17:39:00"
print LST(ut)
print JD(ut)

19:32:23
2457659.23542


In [9]:
KAT.date=now()
print LST(KAT.date)

17:08:17


In [10]:
str(localtime(now()))

'2016-09-27 17:15:17.000001'

In [11]:
now()

42639.135613425926

In [12]:
JD(ut)-now()-JD("1899/12/31 12:00:00")

0.09980324050411582

In [13]:
JD("2015/05/06 12:00:00")

2457149.0

In [14]:
print JD("1899/12/31 12:00:00"), LST("1899/12/31 12:00:00")

2415020.0 20:04:25


In [15]:
str(now()) # UT time now

'2016/9/27 15:15:18'

In [16]:
2457149.08403 -42129.084490740745

2415019.9995392594